In [ ]:
import pandas as pd
import os

# Load team and match data
teams = pd.read_csv("new_data/database_teams.csv")
matches = pd.read_csv("new_data/database_matches.csv")


In [ ]:
# Merge home team stats
matches = matches.merge(
    teams.add_prefix("home_"),
    left_on="homeTeamId",
    right_on="home_teamId",
    how="inner"
)

# Merge away team stats
matches = matches.merge(
    teams.add_prefix("away_"),
    left_on="awayTeamId",
    right_on="away_teamId",
    how="inner"
).sort_values(
    by=["date"]
)

def get_result(row):
    if row['homeTeamWinner'] == True:
        return 'HomeWin'
    elif row['awayTeamWinner'] == True:
        return 'AwayWin'
    elif row['homeTeamWinner'] == False and row['awayTeamWinner'] == False:
        return 'Draw'
    else:
        return 'Unknown'

matches['Result'] = matches.apply(get_result, axis=1)

matches.drop(columns=[
    "homeTeamId", "awayTeamId",
    "home_teamId", "away_teamId",
    "homeTeamScore", "awayTeamScore",
    "homeTeamWinner", "awayTeamWinner",
    "leagueId", "home_Games", "away_Games", "away_League", "home_Goal_for", "home_Goal_against", "away_Goal_for", "away_Goal_against", "home_Wins", "home_Ties", "home_Losses", "away_Wins", "away_Ties", "away_Losses", "home_Points", "away_Points",
], inplace=True)

In [ ]:
new_data_dir = "new_data"

output_path_matches = os.path.join(new_data_dir, 'merged_matches.csv')
os.makedirs(os.path.dirname(output_path_matches), exist_ok=True)
matches.to_csv(output_path_matches, index=False)
print(f"Merged CSV saved to {output_path_matches}")